In [1]:
import regex as re
import pandas as pd
from nltk.translate.bleu_score import SmoothingFunction
import random
from datasets import load_dataset

dataset = load_dataset("rajpurkar/squad")
validation = dataset['validation']

random.seed(42)

import pandas as pd
import regex as re
from tqdm import tqdm
import nltk
import RagNRollQA

c:\Users\zisak\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
len(validation['question'])

10570

In [4]:
# get length of questions which start with when and doesn't contain what, who, where, how and save the fake when questions indicies
when_questions = []
fake_when_questions = []
fake_when_questions_indices = []
for index, question in enumerate(validation['question']):
    if question.startswith("When") and not any(word in question.lower() for word in [", what", ", who", ", where", ", how"]):
        when_questions.append(question)
    elif question.startswith("When"):
        fake_when_questions.append(question)
        fake_when_questions_indices.append(index)
        
    
        
# remove rows from validation which contains questions inside fake_when_questions_indices
validation = validation.filter(lambda example, idx: idx not in fake_when_questions_indices, with_indices=True)
fake_when_questions_indices

[260,
 1420,
 5063,
 5788,
 6600,
 6808,
 6885,
 6904,
 7362,
 7459,
 8962,
 9035,
 9599,
 9667,
 9877,
 10253,
 10403,
 10422]

In [5]:
len(validation['question'])

10552

In [6]:
# Text normalization
import re
import string

def remove_articles(text):
    # Removes a, an, the
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

def remove_extra_spaces(text):
    # Removes extra spaces
    # return " ".join(text.split())
    return re.sub(r"\s+", " ", text)

def remove_punctuation(text):
    # Removes punctuation
    punctuation = set(string.punctuation)
    return "".join(ch for ch in text if ch not in punctuation)

def lower_case(text):
    # Converts text to lower case
    return text.lower()

def normalize_text(text):
    return remove_extra_spaces(remove_articles(remove_punctuation(lower_case(text))))

In [7]:
import evaluate
import nltk
from collections import Counter

def ExactMatch(predicted_answer, golden_answer):
    return {'EM' : int(predicted_answer == golden_answer)}

def F1Score(predicted_answer, golden_answer):
    predicted_answer = predicted_answer.split()
    golden_answer = golden_answer.split()

    if len(predicted_answer) == 0 or len(golden_answer) == 0:
        # If either the predicted answer or the golden answer is empty,
        # then the F1 score is 1 if both are empty, and 0 if only one is empty.
        return {'F1' : int(predicted_answer == golden_answer), 'Precision' : int(predicted_answer == golden_answer), 'Recall' : int(predicted_answer == golden_answer)}

    # for example, if 
    # predicted_answer is ['a', 'b', 'b', 'c'] and 
    # golden_answer is ['b', 'b', 'c', 'd'], then 
    # common will be Counter({'b': 2, 'c': 1}) and num_same will be 3.
    common = Counter(predicted_answer) & Counter(golden_answer)
    num_same = sum(common.values())
    
    if num_same == 0:
        # No common words between the predicted answer and the golden answer
        return {'F1' : 0, 'Precision' : 0, 'Recall' : 0}
    
    precision = num_same / len(predicted_answer)        # tp / (tp + fp)
    recall = num_same / len(golden_answer)              # tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall + 0.00000001)
    return {'F1' : f1, 'Precision' : precision, 'Recall' : recall}

def BLEU_Score(predicted_answer, golden_answer, smoothing_function):
    Bleu1 = nltk.translate.bleu_score.sentence_bleu([golden_answer], predicted_answer, weights=(1, 0, 0, 0), smoothing_function=smoothing_function)
    Bleu2 = nltk.translate.bleu_score.sentence_bleu([golden_answer], predicted_answer, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing_function)
    Bleu3 = nltk.translate.bleu_score.sentence_bleu([golden_answer], predicted_answer, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothing_function)
    Bleu4 = nltk.translate.bleu_score.sentence_bleu([golden_answer], predicted_answer, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing_function)
    Bleu = (Bleu1 * Bleu2 * Bleu3 * Bleu4) ** (1/4)     # geometric mean

    # hand-made bleu
    n = min(len(predicted_answer.split()), 4)
    if n == 0:
        BLEU_handmade = 0
    else:
        weights = [1.0/n]*n
        BLEU_handmade = nltk.translate.bleu_score.sentence_bleu([golden_answer], predicted_answer, weights=weights, smoothing_function=smoothing_function)
    
    return {'BLEU' : Bleu, 'BLEU1' : Bleu1, 'BLEU2' : Bleu2, 'BLEU3' : Bleu3, 'BLEU4' : Bleu4, 'BLEU_handmade' : BLEU_handmade}

def BLEU_built_in_Score(predicted_answer, golden_answer, bleu):
    return {'bleu' : bleu.compute(predictions=[predicted_answer], references=[[golden_answer]])['bleu']}
    
def ROUGE_Score(predicted_answer, golden_answer, rouge):
    return rouge.compute(predictions=[predicted_answer], references=[[golden_answer]])

def Meteor_Score(predicted_answer, golden_answer, meteor):
    return {'meteor' : meteor.compute(predictions=[predicted_answer], references=[[golden_answer]])['meteor']}

def GLEU_Score(predicted_answer, golden_answer):
    return {'gleu' : nltk.translate.gleu_score.sentence_gleu([golden_answer], predicted_answer)}

def MetricsEvaluation(predicted_answer, golden_answer, smoothing_function, bleu, rouge, meteor):
    predicted_answer = normalize_text(predicted_answer)
    golden_answer = normalize_text(golden_answer)

    EM = ExactMatch(predicted_answer, golden_answer)
    F1 = F1Score(predicted_answer, golden_answer)

    BLEU = BLEU_Score(predicted_answer, golden_answer, smoothing_function)
    
    # BLEU_bulit_in = BLEU_built_in_Score(predicted_answer, golden_answer, bleu)
    
    ROUGE = ROUGE_Score(predicted_answer, golden_answer, rouge)

    Meteor = Meteor_Score(predicted_answer, golden_answer, meteor)

    GLEU = GLEU_Score(predicted_answer, golden_answer)


    return {**EM, **F1, **BLEU, **ROUGE, **Meteor, **GLEU}



c:\Users\zisak\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [8]:
def print_metrics(metrics, total, file_name):
    metrics = dict(sorted(metrics.items()))
    for key, value in metrics.items():
        print(f"{key}: {100*value/total:.2f}%")
    print(f"\nTotal: {total}")

    total_str = str(total)
    # Open the file in write mode ('w')
    with open(file_name + '.txt', 'w') as f:
        # Write the metrics and total to the file
        for key, value in metrics.items():
            f.write(f"{key}: {100*value/total:.2f}%\n")
        f.write("\nTotal: " + total_str + "\n")

In [24]:
random.randint(1, 100)

36

In [44]:
def Evaluate(validation, smoothing_function, bleu, rouge, meteor, RagQA, starts_with=[]):
    included = [7, 11, 17, 22, 33, 34, 55, 92, 126, 137, 140, 146, 169, 205, 338, 518, 573, 577, 579, 632, 664, 700, 747, 751, 833, 850, 877, 888, 914, 919, 928, 935, 949, 974, 979, 987, 1014, 1019, 1100, 1135, 1194, 1199, 1221, 1229, 1232, 1235, 1238, 1242, 1248, 1258, 1262, 1264, 1317, 1324, 1338, 1368, 1371, 1395, 1405, 1406, 1408, 1411, 1412, 1436, 1468, 1471, 1476, 1487, 1520, 1551, 1552, 1580, 1588, 1598, 1599, 1603, 1686, 1687, 1688, 1709, 1711, 1744, 1770, 1771, 1803, 1835, 1843, 1848, 1856, 1916, 1926, 1947, 1963, 1973, 1978, 1990, 1993, 1994, 2008, 2010, 2013, 2019, 2030, 2052, 2053, 2061, 2062, 2069, 2086, 2089, 2113, 2143, 2158, 2193, 2198, 2209, 2252, 2276, 2280, 2288, 2292, 2343, 2359, 2367, 2370, 2374, 2388, 2389, 2406, 2415, 2416, 2437, 2438, 2446, 2485, 2487, 2493, 2512, 2523, 2535, 2539, 2563, 2573, 2577, 2696, 2701, 2732, 2764, 2808, 2827, 2828, 2847, 2895, 2902, 2903, 2920, 2957, 3018, 3039, 3080, 3094, 3096, 3119, 3128, 3146, 3150, 3152, 3220, 3231, 3234, 3284, 3289, 3309, 3310, 3337, 3350, 3351, 3385, 3391, 3417, 3429, 3433, 3437, 3443, 3485, 3492, 3504, 3542, 3544, 3562, 3574, 3585, 3599, 3614, 3621, 3634, 3649, 3650, 3667, 3683, 3822, 3856, 3870, 3898, 3900, 3911, 3914, 3926, 3935, 3955, 3959, 3968, 3974, 3994, 4007, 4023, 4027, 4033, 4062, 4063, 4064, 4067, 4069, 4071, 4072, 4088, 4098, 4131, 4173, 4178, 4180, 4186, 4209, 4232, 4244, 4257, 4273, 4283, 4284, 4298, 4299, 4309, 4351, 4405, 4491, 4532, 4557, 4564, 4594, 4623, 4641, 4649, 4673, 4683, 4768, 4782, 4789, 4811, 4837, 4840, 4841, 4848, 4852, 4857, 4862, 4865, 4876, 4889, 4892, 4905, 4917, 4924, 4948, 4949, 4953, 4954, 4975, 4977, 4989, 5002, 5080, 5101, 5103, 5143, 5195, 5199, 5209, 5234, 5258, 5264, 5267, 5302, 5305, 5329, 5336, 5338, 5343, 5349, 5356, 5366, 5385, 5394, 5435, 5474, 5478, 5541, 5543, 5677, 5686, 5711, 5720, 5727, 5743, 5792, 5844, 5856, 5877, 5882, 5912, 5931, 5947, 5954, 5995, 5997, 6013, 6023, 6037, 6096, 6103, 6106, 6112, 6161, 6164, 6165, 6179, 6198, 6220, 6234, 6279, 6283, 6292, 6305, 6312, 6339, 6346, 6347, 6349, 6360, 6391, 6394, 6435, 6449, 6459, 6461, 6463, 6466, 6469, 6470, 6474, 6478, 6494, 6517, 6615, 6660, 6670, 6692, 6699, 6738, 6773, 6794, 6840, 6841, 6864, 6870, 6920, 6924, 6927, 6969, 7018, 7035, 7051, 7133, 7140, 7169, 7175, 7188, 7241, 7282, 7307, 7349, 7350, 7361, 7368, 7381, 7404, 7409, 7415, 7469, 7470, 7489, 7500, 7515, 7525, 7529, 7538, 7540, 7557, 7559, 7565, 7572, 7580, 7585, 7600, 7615, 7685, 7755, 7802, 7805, 7844, 7850, 7862, 7874, 7897, 8001, 8021, 8029, 8126, 8191, 8243, 8277, 8280, 8287, 8346, 8359, 8363, 8378, 8393, 8402, 8403, 8408, 8415, 8420, 8429, 8449, 8464, 8473, 8494, 8497, 8528, 8529, 8531, 8532, 8549, 8561, 8566, 8570, 8571, 8572, 8575, 8585, 8614, 8629, 8630, 8636, 8656, 8694, 8715, 8719, 8744, 8747, 8758, 8762, 8766, 8781, 8797, 8798, 8799, 8802, 8806, 8819, 8828, 8830, 8855, 8856, 8857, 8862, 8865, 8869, 8874, 8878, 8916, 8921, 8923, 8930, 8938, 8945, 8947, 9010, 9025, 9047, 9050, 9053, 9098, 9152, 9168, 9182, 9225, 9261, 9331, 9405, 9407, 9409, 9427, 9483, 9488, 9508, 9516, 9522, 9534, 9541, 9568, 9577, 9594, 9597, 9610, 9618, 9620, 9629, 9632, 9649, 9656, 9681, 9682, 9709, 9726, 9738, 9750, 9767, 9775, 9988, 9991, 10002, 10041, 10046, 10061, 10062, 10081, 10088, 10094, 10097, 10104, 10142, 10146, 10195, 10199, 10212, 10218, 10227, 10246, 10262, 10271, 10300, 10305, 10332, 10348, 10381, 10394, 10416, 10435, 10478, 10517, 10536, 10541]    
    total = 0
    errors = []
    corrects = []
    metrics = {}
    counter = 0
    for record in tqdm(validation):
        try:
            counter += 1
            if counter % 1000 == 0 and counter != 0:
                print(f"metrics: {metrics}")
                print(f"Total: {total}")
            if len(starts_with) > 0 and not any(record['question'].lower().startswith(start) for start in starts_with):
                continue
            # make uniform distributed int from 1 to 100   
            random_int = random.randint(1, 100)
            if counter in included or random_int < 8:
                total += 1
            # if counter != 1624:
            #     continue
            # else:
            #     pass
                predicted_answer = RagQA.answer_question(record['question'], record['context'])
                metrics_instance = MetricsEvaluation(predicted_answer, record['answers']['text'][0], smoothing_function, bleu, rouge, meteor)
                metrics = {k: metrics.get(k, 0) + metrics_instance.get(k, 0) for k in set(metrics) | set(metrics_instance)}
                # if metrics_instance["F1"] > 0.5:
                #     corrects.append(counter)
        except Exception as e:
            errors.append(counter)
            print(f"Error at {counter}: {e} ")
            continue
    return metrics, total, corrects, errors

In [26]:
smoothing_function = nltk.translate.bleu_score.SmoothingFunction().method4
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zisak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zisak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\zisak\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
RagQA = RagNRollQA.RagNRollQA()

c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [39]:
import time

context = "the surface of the Moon occurred several seconds after 20:17:40 UTC on July 20, 1969. One of three probes attached to three of the lunar module's four legs made contact with the surface, a panel light in the LM illuminated, and Aldrin called out, \"Contact light.\" Armstrong shut the engine off and said, \"Shutdown.\" As the LM settled onto the surface, Aldrin said, \"Okay, engine stop\"; then they both called out some post-landing checklist items. After a ten-second pause, Duke acknowledged the landing with, \"We copy you down, \"Eagle\".\" Armstrong announced the landing to Mission Control and the world with"
question = "when did Neil Armstrong land on Moon."
start = time.time()
RagQA.answer_question(question, context)
end = time.time()
print(f"Time taken: {end-start}")

Time taken: 0.24888324737548828


In [45]:
RagQA = RagNRollQA.RagNRollQA()
startWith = "what"
metrics, total, corrects, errors = Evaluate(validation, smoothing_function, bleu, rouge, meteor, RagQA, starts_with=[startWith+' '])
file_name = f"results/Classical_{startWith}" + str(total) + ".txt"
print_metrics(metrics, total, file_name)
print(f"Corrects: {corrects}")
print(f"Errors: {errors}")

c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  9%|▊         | 914/10552 [00:41<08:38, 18.58it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
  9%|▉         | 995/10552 [00:48<11:45, 13.55it/s]

metrics: {'BLEU4': 28.488975488015807, 'BLEU_handmade': 29.374804768805195, 'BLEU': 29.847664329135885, 'BLEU3': 29.317824547586746, 'BLEU2': 30.57060992702669, 'Precision': 29.263412698412697, 'rougeLsum': 32.22814439851792, 'rouge1': 32.22814439851792, 'rouge2': 18.04824284910492, 'BLEU1': 32.90072563713947, 'EM': 16, 'Recall': 40.666666666666664, 'meteor': 28.83563748402777, 'rougeL': 32.22814439851792, 'F1': 32.38919426731101, 'gleu': 29.393562086265103}
Total: 57


 19%|█▉        | 1995/10552 [01:59<25:13,  5.65it/s]

metrics: {'BLEU4': 75.16450310251037, 'BLEU_handmade': 77.32686203330047, 'BLEU': 78.55649245395334, 'BLEU3': 77.24186221666216, 'BLEU2': 80.43186923030396, 'Precision': 79.03941110660925, 'rougeLsum': 85.10917042109192, 'rouge1': 85.22681747991545, 'rouge2': 53.63470972124952, 'BLEU1': 86.01967863571667, 'EM': 34, 'Recall': 104.70357142857144, 'meteor': 78.7370389472843, 'rougeL': 85.10917042109192, 'F1': 85.38786704124811, 'gleu': 77.87774868164757}
Total: 150


 28%|██▊       | 2987/10552 [03:31<06:30, 19.39it/s]

...

metrics: {'BLEU4': 384.1152406855824, 'BLEU_handmade': 395.83830873218216, 'BLEU': 400.9781846300241, 'BLEU3': 394.70591874666025, 'BLEU2': 409.2482652775283, 'Precision': 404.19296423338244, 'rougeLsum': 433.0419771361048, 'rouge1': 433.6844593597635, 'rouge2': 319.6591302074554, 'BLEU1': 438.704622041231, 'EM': 170, 'Recall': 524.1900829433723, 'meteor': 428.1537604549033, 'rougeL': 433.0419771361048, 'F1': 433.0845057631058, 'gleu': 399.9223923100283}
Total: 732


 92%|█████████▏| 9683/10552 [13:00<01:36,  9.04it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 95%|█████████▍| 9991/10552 [13:17<00:28, 19.88it/s]

metrics: {'BLEU4': 416.22289319829696, 'BLEU_handmade': 430.37033029340984, 'BLEU': 435.0243011295273, 'BLEU3': 427.96942147217874, 'BLEU2': 444.38696128228304, 'Precision': 438.76562698729526, 'rougeLsum': 470.91169620975296, 'rouge1': 471.606810012359, 'rouge2': 345.94237489178914, 'BLEU1': 477.18190380544934, 'EM': 184, 'Recall': 571.3150829433723, 'meteor': 465.9909686210552, 'rougeL': 470.91169620975296, 'F1': 470.992980146393, 'gleu': 434.27162388719205}
Total: 810


100%|██████████| 10552/10552 [14:15<00:00, 12.33it/s]

BLEU: 53.36%
BLEU1: 58.63%
BLEU2: 54.50%
BLEU3: 52.48%
BLEU4: 51.05%
BLEU_handmade: 52.76%
EM: 22.42%
F1: 57.89%
Precision: 54.14%
Recall: 69.94%
gleu: 53.32%
meteor: 57.08%
rouge1: 57.96%
rouge2: 42.50%
rougeL: 57.87%
rougeLsum: 57.87%

Total: 861
Corrects: []
Errors: []


In [40]:
"""
when: [111, 135, 138, 139, 141, 154, 167, 173, 178, 214, 363, 432, 441, 449, 461, 835, 878, 880, 887, 895, 900, 904, 921, 925, 937, 939, 942, 946, 980, 984, 1018, 1042, 1058, 1066, 1086, 1099, 1107, 1111, 1119, 1129, 1156, 1158, 1181, 1186, 1190, 1200, 1224, 1228, 1245, 1254, 1263, 1282, 1294, 1301, 1316, 1328, 1331, 1337, 1341, 1375, 1393, 1396, 1420, 1426, 1430, 1454, 1462, 1488, 1505, 1545, 1556, 1632, 1645, 1651, 1653, 1657, 1669, 2075, 2077, 2085, 2175, 2184, 2187, 2189, 2190, 2194, 2199, 2227, 2250, 2261, 2265, 2267, 2274, 2285, 2296, 2299, 2305, 2314, 2346, 2354, 2384, 2390, 2404, 2405, 2434, 2439, 2472, 2475, 2478, 2550, 2556, 2564, 2576, 2591, 2597, 2600, 2620, 2794, 2818, 2820, 2832, 2849, 2850, 2856, 2861, 2866, 2886, 2898, 2969, 3011, 3015, 3037, 3044, 3046, 3054, 3060, 3074, 3076, 3084, 3125, 3135, 3151, 3166, 3183, 3212, 3215, 3238, 3241, 3584, 3630, 3641, 3645, 3709, 3714, 3722, 3723, 3726, 3729, 3730, 3732, 3738, 3771, 3777, 3778, 3789, 3790, 3794, 3800, 3816, 3842, 3893, 3927, 3975, 3993, 4111, 4136, 4140, 4187, 4201, 4226, 4227, 4243, 4254, 4294, 4340, 4539, 4652, 4668, 4711, 4715, 4748, 4758, 4790, 4853, 4915, 4920, 4931, 4933, 4939, 4960, 4969, 4973, 5003, 5005, 5058, 5183, 5188, 5200, 5207, 5275, 5280, 5357, 5423, 5436, 5577, 5579, 5592, 5699, 5701, 5725, 5728, 5783, 5785, 5788, 5794, 5795, 5817, 5819, 5825, 5836, 5840, 5848, 5885, 5895, 5907, 5908, 5927, 5972, 5977, 6019, 6024, 6049, 6055, 6072, 6149, 6228, 6265, 6405, 6413, 6513, 7117, 7167, 7174, 7256, 7413, 7532, 7555, 7564, 7570, 7669, 7718, 7838, 7870, 7898, 7904, 7908, 7939, 7970, 7978, 8032, 8035, 8036, 8123, 8129, 8152, 8160, 8169, 8174, 8178, 8182, 8192, 8205, 8262, 8274, 8296, 8301, 8303, 8308, 8353, 8355, 8356, 8367, 8370, 8380, 8391, 8394, 8396, 8419, 8427, 8432, 8434, 8442, 8526, 8533, 8534, 8537, 8565, 8587, 8606, 8617, 8618, 8640, 8644, 8648, 8659, 8690, 8776, 8824, 8985, 9055, 9061, 9072, 9226, 9309, 9314, 9346, 9350, 9351, 9356, 9357, 9363, 9374, 9384, 9403, 9406, 9426, 9574, 9625, 9628, 9636, 9655, 9659, 9671, 9677, 9679, 9717, 9720, 9722, 9728, 9739, 9758, 9766, 9776, 9777, 9779, 9782, 9793, 9845, 9871, 9879, 9888, 9906, 9907, 9911, 9932, 9982, 9989, 9996, 10004, 10012, 10016, 10018, 10020, 10029, 10050, 10060, 10165, 10177, 10185, 10197, 10231, 10255, 10261, 10296, 10334, 10338, 10346, 10364, 10465, 10481]
where: [423, 426, 428, 429, 430, 431, 497, 851, 855, 890, 894, 1005, 1109, 1149, 1165, 1183, 1210, 1233, 1281, 1285, 1329, 1363, 1379, 1437, 1440, 1582, 1583, 1586, 1611, 2025, 2029, 2160, 2192, 2220, 2290, 2409, 2453, 2454, 2916, 2918, 3055, 3233, 3382, 3461, 4220, 4341, 4347, 4592, 4624, 4628, 4631, 4647, 4722, 4802, 4912, 4919, 4935, 5067, 5077, 5326, 5379, 5574, 5676, 5741, 5786, 5789, 5806, 6182, 6398, 6409, 6626, 6998, 7024, 7945, 8173, 8384, 8409, 8462, 8474, 8639, 8741, 8764, 8769, 8805, 8829, 8887, 8914, 9086, 9155, 9292, 9330, 9429, 9447, 9608, 9660, 9661, 9664, 9665, 9713, 9735, 9762, 9834, 9835, 9923, 9997, 10116, 10193, 10244, 10245, 10285, 10339, 10424, 10425]
who: [86, 99, 107, 112, 179, 201, 282, 287, 300, 354, 357, 368, 401, 471, 481, 527, 528, 533, 591, 607, 626, 628, 691, 693, 703, 712, 713, 715, 731, 732, 733, 735, 740, 745, 749, 752, 754, 759, 771, 789, 899, 907, 1030, 1088, 1105, 1110, 1118, 1139, 1140, 1146, 1148, 1157, 1167, 1184, 1284, 1297, 1302, 1374, 1383, 1387, 1446, 1460, 1496, 1571, 1578, 1585, 1824, 1831, 1895, 1902, 1905, 1911, 1912, 1934, 1975, 2114, 2129, 2185, 2202, 2206, 2224, 2228, 2264, 2270, 2271, 2278, 2286, 2313, 2320, 2338, 2358, 2476, 2514, 2551, 2557, 2578, 2609, 2698, 2810, 2825, 2877, 3021, 3047, 3217, 3250, 3285, 3348, 3366, 3376, 3405, 3441, 3718, 3733, 3846, 3847, 3852, 3894, 3940, 3997, 4008, 4057, 4075, 4076, 4085, 4089, 4101, 4130, 4133, 4138, 4407, 4411, 4475, 4759, 4887, 4950, 4974, 5014, 5113, 5115, 5121, 5125, 5191, 5288, 5346, 5354, 5451, 5481, 5482, 5611, 5664, 5800, 5909, 5967, 5968, 6121, 6123, 6129, 6133, 6158, 6194, 6208, 6216, 6236, 6266, 6328, 6414, 6465, 6508, 6736, 6746, 6750, 6775, 6835, 6910, 6911, 6957, 6959, 6961, 6981, 6982, 6984, 6989, 7243, 7247, 7255, 7283, 7339, 7513, 7516, 7544, 7625, 7675, 7714, 7774, 7779, 7784, 7789, 7799, 7800, 7813, 7849, 7903, 7906, 7907, 7934, 7950, 7982, 7998, 8003, 8042, 8188, 8190, 8201, 8202, 8295, 8323, 8324, 8374, 8401, 8425, 8436, 8451, 8516, 8525, 8578, 8599, 8691, 8778, 9388, 9392, 9396, 9439, 9442, 9457, 9458, 9461, 9493, 9500, 9535, 9537, 9555, 9558, 9596, 9657, 9662, 9673, 9701, 9721, 9940, 9947, 9969, 10013, 10027, 10032, 10107, 10140, 10141, 10147, 10233, 10256, 10274, 10326, 10341, 10353, 10359, 10371, 10380, 10420, 10421, 10450, 10470]
what: [7, 11, 17, 22, 33, 34, 55, 92, 126, 137, 140, 146, 169, 205, 338, 518, 573, 577, 579, 632, 664, 700, 747, 751, 833, 850, 877, 888, 914, 919, 928, 935, 949, 974, 979, 987, 1014, 1019, 1100, 1135, 1194, 1199, 1221, 1229, 1232, 1235, 1238, 1242, 1248, 1258, 1262, 1264, 1317, 1324, 1338, 1368, 1371, 1395, 1405, 1406, 1408, 1411, 1412, 1436, 1468, 1471, 1476, 1487, 1520, 1551, 1552, 1580, 1588, 1598, 1599, 1603, 1686, 1687, 1688, 1709, 1711, 1744, 1770, 1771, 1803, 1835, 1843, 1848, 1856, 1916, 1926, 1947, 1963, 1973, 1978, 1990, 1993, 1994, 2008, 2010, 2013, 2019, 2030, 2052, 2053, 2061, 2062, 2069, 2086, 2089, 2113, 2143, 2158, 2193, 2198, 2209, 2252, 2276, 2280, 2288, 2292, 2343, 2359, 2367, 2370, 2374, 2388, 2389, 2406, 2415, 2416, 2437, 2438, 2446, 2485, 2487, 2493, 2512, 2523, 2535, 2539, 2563, 2573, 2577, 2696, 2701, 2732, 2764, 2808, 2827, 2828, 2847, 2895, 2902, 2903, 2920, 2957, 3018, 3039, 3080, 3094, 3096, 3119, 3128, 3146, 3150, 3152, 3220, 3231, 3234, 3284, 3289, 3309, 3310, 3337, 3350, 3351, 3385, 3391, 3417, 3429, 3433, 3437, 3443, 3485, 3492, 3504, 3542, 3544, 3562, 3574, 3585, 3599, 3614, 3621, 3634, 3649, 3650, 3667, 3683, 3822, 3856, 3870, 3898, 3900, 3911, 3914, 3926, 3935, 3955, 3959, 3968, 3974, 3994, 4007, 4023, 4027, 4033, 4062, 4063, 4064, 4067, 4069, 4071, 4072, 4088, 4098, 4131, 4173, 4178, 4180, 4186, 4209, 4232, 4244, 4257, 4273, 4283, 4284, 4298, 4299, 4309, 4351, 4405, 4491, 4532, 4557, 4564, 4594, 4623, 4641, 4649, 4673, 4683, 4768, 4782, 4789, 4811, 4837, 4840, 4841, 4848, 4852, 4857, 4862, 4865, 4876, 4889, 4892, 4905, 4917, 4924, 4948, 4949, 4953, 4954, 4975, 4977, 4989, 5002, 5080, 5101, 5103, 5143, 5195, 5199, 5209, 5234, 5258, 5264, 5267, 5302, 5305, 5329, 5336, 5338, 5343, 5349, 5356, 5366, 5385, 5394, 5435, 5474, 5478, 5541, 5543, 5677, 5686, 5711, 5720, 5727, 5743, 5792, 5844, 5856, 5877, 5882, 5912, 5931, 5947, 5954, 5995, 5997, 6013, 6023, 6037, 6096, 6103, 6106, 6112, 6161, 6164, 6165, 6179, 6198, 6220, 6234, 6279, 6283, 6292, 6305, 6312, 6339, 6346, 6347, 6349, 6360, 6391, 6394, 6435, 6449, 6459, 6461, 6463, 6466, 6469, 6470, 6474, 6478, 6494, 6517, 6615, 6660, 6670, 6692, 6699, 6738, 6773, 6794, 6840, 6841, 6864, 6870, 6920, 6924, 6927, 6969, 7018, 7035, 7051, 7133, 7140, 7169, 7175, 7188, 7241, 7282, 7307, 7349, 7350, 7361, 7368, 7381, 7404, 7409, 7415, 7469, 7470, 7489, 7500, 7515, 7525, 7529, 7538, 7540, 7557, 7559, 7565, 7572, 7580, 7585, 7600, 7615, 7685, 7755, 7802, 7805, 7844, 7850, 7862, 7874, 7897, 8001, 8021, 8029, 8126, 8191, 8243, 8277, 8280, 8287, 8346, 8359, 8363, 8378, 8393, 8402, 8403, 8408, 8415, 8420, 8429, 8449, 8464, 8473, 8494, 8497, 8528, 8529, 8531, 8532, 8549, 8561, 8566, 8570, 8571, 8572, 8575, 8585, 8614, 8629, 8630, 8636, 8656, 8694, 8715, 8719, 8744, 8747, 8758, 8762, 8766, 8781, 8797, 8798, 8799, 8802, 8806, 8819, 8828, 8830, 8855, 8856, 8857, 8862, 8865, 8869, 8874, 8878, 8916, 8921, 8923, 8930, 8938, 8945, 8947, 9010, 9025, 9047, 9050, 9053, 9098, 9152, 9168, 9182, 9225, 9261, 9331, 9405, 9407, 9409, 9427, 9483, 9488, 9508, 9516, 9522, 9534, 9541, 9568, 9577, 9594, 9597, 9610, 9618, 9620, 9629, 9632, 9649, 9656, 9681, 9682, 9709, 9726, 9738, 9750, 9767, 9775, 9988, 9991, 10002, 10041, 10046, 10061, 10062, 10081, 10088, 10094, 10097, 10104, 10142, 10146, 10195, 10199, 10212, 10218, 10227, 10246, 10262, 10271, 10300, 10305, 10332, 10348, 10381, 10394, 10416, 10435, 10478, 10517, 10536, 10541]
""";

In [42]:
len([7, 11, 17, 22, 33, 34, 55, 92, 126, 137, 140, 146, 169, 205, 338, 518, 573, 577, 579, 632, 664, 700, 747, 751, 833, 850, 877, 888, 914, 919, 928, 935, 949, 974, 979, 987, 1014, 1019, 1100, 1135, 1194, 1199, 1221, 1229, 1232, 1235, 1238, 1242, 1248, 1258, 1262, 1264, 1317, 1324, 1338, 1368, 1371, 1395, 1405, 1406, 1408, 1411, 1412, 1436, 1468, 1471, 1476, 1487, 1520, 1551, 1552, 1580, 1588, 1598, 1599, 1603, 1686, 1687, 1688, 1709, 1711, 1744, 1770, 1771, 1803, 1835, 1843, 1848, 1856, 1916, 1926, 1947, 1963, 1973, 1978, 1990, 1993, 1994, 2008, 2010, 2013, 2019, 2030, 2052, 2053, 2061, 2062, 2069, 2086, 2089, 2113, 2143, 2158, 2193, 2198, 2209, 2252, 2276, 2280, 2288, 2292, 2343, 2359, 2367, 2370, 2374, 2388, 2389, 2406, 2415, 2416, 2437, 2438, 2446, 2485, 2487, 2493, 2512, 2523, 2535, 2539, 2563, 2573, 2577, 2696, 2701, 2732, 2764, 2808, 2827, 2828, 2847, 2895, 2902, 2903, 2920, 2957, 3018, 3039, 3080, 3094, 3096, 3119, 3128, 3146, 3150, 3152, 3220, 3231, 3234, 3284, 3289, 3309, 3310, 3337, 3350, 3351, 3385, 3391, 3417, 3429, 3433, 3437, 3443, 3485, 3492, 3504, 3542, 3544, 3562, 3574, 3585, 3599, 3614, 3621, 3634, 3649, 3650, 3667, 3683, 3822, 3856, 3870, 3898, 3900, 3911, 3914, 3926, 3935, 3955, 3959, 3968, 3974, 3994, 4007, 4023, 4027, 4033, 4062, 4063, 4064, 4067, 4069, 4071, 4072, 4088, 4098, 4131, 4173, 4178, 4180, 4186, 4209, 4232, 4244, 4257, 4273, 4283, 4284, 4298, 4299, 4309, 4351, 4405, 4491, 4532, 4557, 4564, 4594, 4623, 4641, 4649, 4673, 4683, 4768, 4782, 4789, 4811, 4837, 4840, 4841, 4848, 4852, 4857, 4862, 4865, 4876, 4889, 4892, 4905, 4917, 4924, 4948, 4949, 4953, 4954, 4975, 4977, 4989, 5002, 5080, 5101, 5103, 5143, 5195, 5199, 5209, 5234, 5258, 5264, 5267, 5302, 5305, 5329, 5336, 5338, 5343, 5349, 5356, 5366, 5385, 5394, 5435, 5474, 5478, 5541, 5543, 5677, 5686, 5711, 5720, 5727, 5743, 5792, 5844, 5856, 5877, 5882, 5912, 5931, 5947, 5954, 5995, 5997, 6013, 6023, 6037, 6096, 6103, 6106, 6112, 6161, 6164, 6165, 6179, 6198, 6220, 6234, 6279, 6283, 6292, 6305, 6312, 6339, 6346, 6347, 6349, 6360, 6391, 6394, 6435, 6449, 6459, 6461, 6463, 6466, 6469, 6470, 6474, 6478, 6494, 6517, 6615, 6660, 6670, 6692, 6699, 6738, 6773, 6794, 6840, 6841, 6864, 6870, 6920, 6924, 6927, 6969, 7018, 7035, 7051, 7133, 7140, 7169, 7175, 7188, 7241, 7282, 7307, 7349, 7350, 7361, 7368, 7381, 7404, 7409, 7415, 7469, 7470, 7489, 7500, 7515, 7525, 7529, 7538, 7540, 7557, 7559, 7565, 7572, 7580, 7585, 7600, 7615, 7685, 7755, 7802, 7805, 7844, 7850, 7862, 7874, 7897, 8001, 8021, 8029, 8126, 8191, 8243, 8277, 8280, 8287, 8346, 8359, 8363, 8378, 8393, 8402, 8403, 8408, 8415, 8420, 8429, 8449, 8464, 8473, 8494, 8497, 8528, 8529, 8531, 8532, 8549, 8561, 8566, 8570, 8571, 8572, 8575, 8585, 8614, 8629, 8630, 8636, 8656, 8694, 8715, 8719, 8744, 8747, 8758, 8762, 8766, 8781, 8797, 8798, 8799, 8802, 8806, 8819, 8828, 8830, 8855, 8856, 8857, 8862, 8865, 8869, 8874, 8878, 8916, 8921, 8923, 8930, 8938, 8945, 8947, 9010, 9025, 9047, 9050, 9053, 9098, 9152, 9168, 9182, 9225, 9261, 9331, 9405, 9407, 9409, 9427, 9483, 9488, 9508, 9516, 9522, 9534, 9541, 9568, 9577, 9594, 9597, 9610, 9618, 9620, 9629, 9632, 9649, 9656, 9681, 9682, 9709, 9726, 9738, 9750, 9767, 9775, 9988, 9991, 10002, 10041, 10046, 10061, 10062, 10081, 10088, 10094, 10097, 10104, 10142, 10146, 10195, 10199, 10212, 10218, 10227, 10246, 10262, 10271, 10300, 10305, 10332, 10348, 10381, 10394, 10416, 10435, 10478, 10517, 10536, 10541])

573

In [14]:

def QuestionStartsWith_Accuracy(dataset, startsWith, RagQA):

    correct = 0
    EM = 0
    BLEU = 0
    BLEU1 = 0
    BLEU2 = 0
    BLEU3 = 0
    BLEU4 = 0
    total = 0
    errors = []
    corrects = []
    empties = []
    kolo = 0
    for item in tqdm(dataset):
        random_number = random.randint(0, len(dataset))
        kolo += 1
        included_question = False
        try:
            context = item['context']
            context = re.sub(' +', ' ', context)
            
            question = item['question']
            tempQuestion = question.lower()
            question = re.sub(' +', ' ', question)
            
            answer = item['answers']['text'][0]
            title = item['title']
    
            # check if question starts with startsWith
            for start in startsWith:
                if tempQuestion.startswith(start):
                    included_question = True
                    break
            
            if included_question:
    
                total += 1
                outputAnswer = RagQA.answer_question(question, context)
                if outputAnswer == "":
                    empties.append(kolo)
                    outputAnswer = "No_Answer_Found"
                
                # print("Question: " , question)
                # print("Answer: ", answer, "-------" , "Our Answer: ", outputAnswer)
                if outputAnswer in answer or answer in outputAnswer:
                    correct += 1
                    corrects.append(kolo)
                else:
                    # write to when_wrong_answers.txt
                    with open("results/where_wrong_answers.txt", "a") as file:
                        file.write(f"Question: {question}\n")
                        file.write(f"Answer: {answer}\n")
                        file.write(f"Our Answer: {outputAnswer}\n\n")
                        
                        
                if outputAnswer == answer:
                    EM += 1

                n = min(len(outputAnswer.split()), 4)
                if n == 0:
                    BLEUscore = 0
                else:
                    weights = [1.0/n]*n
                    smoothie = SmoothingFunction().method4
                    BLEUscore = nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=weights, smoothing_function=smoothie)
                    
                    BLEU += BLEUscore
                    BLEU1 += nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=(1, 0, 0, 0), smoothing_function=smoothie)
                    BLEU2 += nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
                    BLEU3 += nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
                    BLEU4 += nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
                    

            if kolo % 1000 == 0:
                print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
                print(f"EM: {EM}, out of {total}: {100*EM/total}%")
                print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
                print(f"BLEU1: {BLEU1}, out of {total}: {100*BLEU1/total}%")
                print(f"BLEU2: {BLEU2}, out of {total}: {100*BLEU2/total}%")
                print(f"BLEU3: {BLEU3}, out of {total}: {100*BLEU3/total}%")
                print(f"BLEU4: {BLEU4}, out of {total}: {100*BLEU4/total}%")
        
        except Exception as e:
            print("title: ", title)
            print("Error in question number: ", total)
            print("Question: ", question)
            print("Answer: ", answer)
            print("Error: ", e)
            errors.append(total)
            # total -= 1
            print("\n\n")
    
    if total != 0:
        print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
        print(f"EM: {EM}, out of {total}: {100*EM/total}%")
        print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
        print(f"BLEU1: {BLEU1}, out of {total}: {100*BLEU1/total}%")
        print(f"BLEU2: {BLEU2}, out of {total}: {100*BLEU2/total}%")
        print(f"BLEU3: {BLEU3}, out of {total}: {100*BLEU3/total}%")
        print(f"BLEU4: {BLEU4}, out of {total}: {100*BLEU4/total}%")
    else:
        print("No Questions found with the given starting word")
    print("Errors: ", errors)
    print("Empties: ", len(empties), empties)
    return correct, total, errors, corrects


if __name__ == "__main__":

    startsWith = ["what "]
    RagQA = RagNRollQA.RagNRollQA()
    x = QuestionStartsWith_Accuracy(validation, startsWith, RagQA)

c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  8%|▊         | 858/10552 [03:44<28:56,  5.58it/s]  c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
  8%|▊         | 859/10552 [03:45<43:15,  3.73it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
  8%|▊         | 862/10552 [03:47<55:53,  2.89it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referre

Correct: 77, out of 368: 20.92391304347826%
EM: 14, out of 368: 3.8043478260869565%
BLEU: 50.10394534184062, out of 368: 13.615202538543645%
BLEU1: 90.97603657741172, out of 368: 24.72174906994884%
BLEU2: 61.55881156514082, out of 368: 16.727937925310005%
BLEU3: 50.252298356225985, out of 368: 13.655515857670105%
BLEU4: 43.7956389195107, out of 368: 11.90098883682356%


 10%|█         | 1056/10552 [04:53<37:41,  4.20it/s]  c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 10%|█         | 1061/10552 [04:53<27:21,  5.78it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 15%|█▌        | 1613/10552 [07:54<40:39,  3.66it/s]  c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 15%|█▌        | 1614/10552 [07:56<55:01,  2.71it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(ref

Correct: 210, out of 852: 24.64788732394366%
EM: 31, out of 852: 3.6384976525821595%
BLEU: 136.20330798339424, out of 852: 15.986303753919511%
BLEU1: 236.80154333623554, out of 852: 27.7936083727976%
BLEU2: 166.43741699245297, out of 852: 19.534908097705745%
BLEU3: 137.01794993471847, out of 852: 16.081919006422357%
BLEU4: 120.39395260817906, out of 852: 14.130745611288622%


 24%|██▎       | 2496/10552 [14:26<4:12:40,  1.88s/it]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 24%|██▎       | 2498/10552 [14:27<3:20:51,  1.50s/it]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 24%|██▎       | 2506/10552 [14:31<46:38,  2.88it/s]  


KeyboardInterrupt: 

In [ ]:
# [174, 315, 326, 349, 472, 488, 493, 498, 530, 667]
# when: 48%